## Spotify Interactive - Beta
This interactive notebook will walk you through the process of exploring your music intake as well as training an algorithm to correctly predict music that you will enjoy. To interact with this notebook you will need to have all of the required libraries downloaded and installed as well as Python. In addition, the user will need to sign up for Spotify for Developers here: https://developer.spotify.com/dashboard/login  You do not need to have Spotify Premium to sign up.

In [1]:
%matplotlib inline
import json
import numpy as np
import pandas as pd
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

From your Spotify dashboard, get your username, client id, secret client id, and redirect website link. The redirct link can be made by going to Settings - Redirect URIs - Add. Add a simple redirect like a localhost. 

In [ ]:
username=""
cid ="" 
secret = ""
redirect=""

Input the scope you are interested in here. Scopes define the kind of query that you are going to make into your data. All of the scope options can be found here: https://developer.spotify.com/documentation/general/guides/scopes/#user-read-recently-played

In [ ]:
# Suggested scope: playlist-read-private
scope = ""

The next step is to authorize access to your backend Spotify data. Start by running the following code and then inserting the output into the variable `token`.

In [ ]:
util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect)

In [ ]:
token = ""
sp = spotipy.Spotify(auth=token)

Next we will choose the playlist that you want to use to explore your music tastes. Run the below code and insert into the variable `playlist` the name of the playlist you are interested in exploring. You can add the variabl `offset` with the desired offset number if you hvae more than 50 playlists. The printed result will be the playlist name and the playlist id.

In [ ]:
playlist = ""
playlists = sp.current_user_playlists()
for i in playlists['items']:
    if i['name'] == playlist:
        print(i['name'])
        print(i['id'])

Now let's explore the songs in the playlist that you chose. Plug the playlist id into the `playlist_id` variable and run the code. The output will list every song in the playlist that you chose with that song's Spotify id.

In [ ]:
songs = sp.user_playlist(username, playlist_id="")
for i in songs['tracks']['items']:
    print('track: ' + i['track']['name'])
    print('id: ' + i['track']['id'])

As a warmup, let's calculate how hipster your playlist is. Aka what is the average popolarity of the songs on your selected playist on a scale of 1 to 100.

In [ ]:
popularity = []
for i in songs['tracks']['items']:
    popularity.append(i['track']['popularity'])
median = np.median(popularity)
mean = np.mean(popularity)
print(f'Popularity median: {median}')
print(f'Popularity mean: {mean}')
pd.Series(popularity).hist(bins=10)

Alright, with that out of the way, we are going to take a three step approach to collecting data about the songs in your playlist and creating a personalizes algorithm.

Step 1 is to collect a large sample of potential recommended songs based off of Spotify's song recommendation API. We will do this based off of random combinations of songs and artitsts in your submitted playlist, combined with a random pool of artists and songs generated from the recommendations of that playlist.

In [ ]:
(track_ids, track_names, artist_ids, artist_names, rec_ids) = ([] for i in range(5))

In [ ]:
for i in songs['tracks']['items']:
    track_ids.append(i['track']['id'])
    track_names.append(i['track']['name'])
    for z in i['track']['artists']:
        artist_ids.append(z['id'])
        artist_names.append(z['name'])

In [ ]:
def get_recs_from_seed(rng):
    print(rng)
    tracks = random.choices(track_ids, k=2)
    artists = random.choices(artist_ids, k=2)
    try:
        rec_list = sp.recommendations(seed_tracks=tracks, seed_artists=artists, limit=10)
        return [i['id'] for i in rec_list['tracks']]
    except:
        rec_list = [] 
        return rec_list

In [ ]:
def get_recs_from_gen(rng):
    print(rng)
    tracks = random.choices(rec_ids, k=5)
    rec_list = sp.recommendations(seed_tracks=tracks, limit=10)
    try:
        rec_list = sp.recommendations(seed_tracks=tracks, seed_artists=artists, limit=10)
        return [i['id'] for i in rec_list['tracks']]
    except:
        rec_list = [] 
        return rec_list

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    for ids in executor.map(get_recs_from_seed, range(10000)):
        rec_ids.extend(ids)
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    for ids in executor.map(get_recs_from_gen, range(10000)):
        rec_ids.extend(ids)

Step 2 is the most computationally intensive and will look to paint a picture of each song in your playlist. These literal pictures will be the foundation of our unsupervised learning algorithm. We will start to paint these pictures by retrieving information about each section and segment within a song. A section is defined by large variations in rhythm or timbre, e.g. chorus, verse, bridge, guitar solo, etc. Each section contains its own descriptions of tempo, key, mode, time_signature, and loudness. A segment attempts to subdivide a song into many segments, with each segment containing a roughly consitent sound throughout its duration.

Let's explore these concepts. Take a spotify id for a song that you found above and plug it into the variable song_id. We will look at the loudness of the song's sections and segments. Since segments are extremely short periods of time (a song will have hundreds to thousands of segments), we will cut the first half percent and back one percent of the song to avoid picking up the lack of sound before a song starts or when a song ends.

In [ ]:
song_id = ''
audio_object = sp.audio_analysis(id=song_id)
segment_loudness = [i['loudness_max'] for i in audio_object['segments']]
half_percent = round(len(segment_loudness)/200)
back_half = len(audio_object['segments']) - half_percent*2
segment_loudness = segment_loudness[half_percent:back_half]
pd.Series(segment_loudness).plot()

In [ ]:
section_loudness = [i['loudness'] for i in audio_object['sections']]
pd.Series(section_loudness).plot()